In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import numpy as np
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm

import sys
sys.path.append('../')

from Configs import getSubAgentConfig, visualizeSubAgentConfig
from Configs import getDatasetConfig, visualizeDatasetConfig
from Environment.EnvironmentSim import createEnv
from Agents.ModelBasedSolvers import MdpKernel
from Environment.PolicySimulator import PolicySimulator
from Agents.Others import RandomPolicy

In [ ]:
def genDatasetByPolicies(
        policies, userMap, num_windows,
        env,
        N_episodes = 10,
        obvMode = "predicted",
        mode="train",
        type="data"
    ):
    len_episode = int(num_windows/N_episodes)
    simResult = {'uRecord': [], 'actionsRecord': [], 'rewardRecord': [], 'uNextRecord': []}
    for i in range(N_episodes):
        policySimulator = PolicySimulator(env)
        policySimulator.loadPolicies(policies, userMap)
        policySimulator.setupModes(obvMode=obvMode, mode=mode, type=type)
        simResult_ep = policySimulator.runSimulation(num_windows=len_episode, N_episodes=N_episodes)
        simResult['actionsRecord'] += simResult_ep['actionsRecord']
        simResult['rewardRecord'] += simResult_ep['rewardRecord']
        simResult['uNextRecord'] += simResult_ep['uNextRecord']
        simResult['uRecord'] += simResult_ep['uRecord']
    return simResult

def load_mdp_policy(configIdx):
    with open(f'../Agents/Agents/MdpPolicy/mdp_config{configIdx}.pkl', 'rb') as f:
        mdpKernelParams = pickle.load(f)
    mdpPolicy = MdpKernel()
    mdpPolicy.load_policy(mdpKernelParams, policyMode="deterministic", randomR=False)
    return mdpPolicy

In [ ]:
num_windows = 10000
N_episodes = 10
len_episode = int(num_windows/N_episodes)
obvMode = "predicted" # "perfect" or "predicted"
savePath = "OfflineDataset"

for datasetConfigIdx in [0,1,2,3]:
    datasetParams = getDatasetConfig(datasetConfigIdx)
    visualizeDatasetConfig(datasetParams)
    trafficDataParentPath = f'TrafficDataset/TrafficData'

    env = createEnv(datasetParams, trafficDataParentPath)

    usermap = datasetParams['user_map']
    poliocy_set = datasetParams['sub_agents_idx']
    simResultTotal = {'uRecord': [], 'actionsRecord': [], 'rewardRecord': [], 'uNextRecord': []}
    for sub_agent_idxs in poliocy_set:
        policies = []
        print(f"sub_agent_idxs: {sub_agent_idxs}")
        for sub_agent_idx in sub_agent_idxs:
            subAgentParams = getSubAgentConfig(sub_agent_idx)
            visualizeSubAgentConfig(subAgentParams) 
            policies.append(load_mdp_policy(sub_agent_idx))

        simResult = genDatasetByPolicies(
            policies, usermap, num_windows, 
            env, 
            N_episodes = N_episodes, obvMode = obvMode, mode="train", type="data")
        simResultTotal['uRecord'] += simResult['uRecord']
        simResultTotal['actionsRecord'] += simResult['actionsRecord']
        simResultTotal['rewardRecord'] += simResult['rewardRecord']
        simResultTotal['uNextRecord'] += simResult['uNextRecord']
        
    print(f"Avg. packet loss rate: {np.mean(simResultTotal['rewardRecord'])}")
    print(f"length of dataset: {len(simResultTotal['uRecord'])}")
    with open(f'{savePath}/subOptimalAgent_encConfig{datasetConfigIdx}_{poliocy_set}.pkl', 'wb') as f:
        pickle.dump(simResultTotal, f)
    print(f"saved to {savePath}/subOptimalAgent_encConfig{datasetConfigIdx}_{poliocy_set}.pkl")

Dataset Configuration
Number of Users:        8
Window Length:          200
N_aggregation:          4
Dataflow:               thumb_fr
Random Seed:            999
Resource Bar:           5
Bandwidth:              100
Sigmoid K List:         [0.1, 0.2, 0.3, 0.4, 0.5]
Sigmoid S List:         [10.0, 10.0, 10.0, 10.0, 10.0, 10.0]
Sub Agents:             [[0, 0]]
User Map:               [[0, 1, 2, 3], [4, 5, 6, 7]]
sub_agent_idxs: [0, 0]
Environment Configuration
Number of Users:        4
Window Length:          200
Dataflow:               thumb_fr
N_aggregation:          4
N_r:                    5
Resource Bar:           5
Bandwidth:              60
Sigmoid K List:         [0.3]
Sigmoid S List:         [10.0]
Random Seed:            999
Environment Configuration
Number of Users:        4
Window Length:          200
Dataflow:               thumb_fr
N_aggregation:          4
N_r:                    5
Resource Bar:           5
Bandwidth:              60
Sigmoid K List:         [0.3]
Sigmoid 

Avg. packet loss rate: 0.3320853710493872
length of dataset: 10000
saved to OfflineDataset/subOptimalAgent_encConfig2_[[0, 0]].pkl
Dataset Configuration
Number of Users:        20
Window Length:          200
N_aggregation:          4
Dataflow:               thumb_fr
Random Seed:            999
Resource Bar:           5
Bandwidth:              200
Sigmoid K List:         [0.1, 0.2, 0.3, 0.4, 0.5]
Sigmoid S List:         [10.0, 10.0, 10.0, 10.0, 10.0, 10.0]
Sub Agents:             [[0, 0, 0, 0, 0]]
User Map:               [[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11], [12, 13, 14, 15], [16, 17, 18, 19]]
sub_agent_idxs: [0, 0, 0, 0, 0]
Environment Configuration
Number of Users:        4
Window Length:          200
Dataflow:               thumb_fr
N_aggregation:          4
N_r:                    5
Resource Bar:           5
Bandwidth:              60
Sigmoid K List:         [0.3]
Sigmoid S List:         [10.0]
Random Seed:            999
Environment Configuration
Number of Users:        4
Wi

Avg. packet loss rate: 0.45171312146306747
length of dataset: 10000
saved to OfflineDataset/subOptimalAgent_encConfig3_[[0, 0, 0, 0, 0]].pkl
Dataset Configuration
Number of Users:        8
Window Length:          200
N_aggregation:          4
Dataflow:               thumb_bk
Random Seed:            999
Resource Bar:           5
Bandwidth:              100
Sigmoid K List:         [0.1, 0.2, 0.3, 0.4, 0.5]
Sigmoid S List:         [10.0, 10.0, 10.0, 10.0, 10.0, 10.0]
Sub Agents:             [[1, 1]]
User Map:               [[0, 1, 2, 3], [4, 5, 6, 7]]
sub_agent_idxs: [1, 1]
Environment Configuration
Number of Users:        4
Window Length:          200
Dataflow:               thumb_bk
N_aggregation:          4
N_r:                    5
Resource Bar:           5
Bandwidth:              60
Sigmoid K List:         [0.3]
Sigmoid S List:         [10.0]
Random Seed:            999
Environment Configuration
Number of Users:        4
Window Length:          200
Dataflow:               thumb_bk
N_

Avg. packet loss rate: 0.3577040989636521
length of dataset: 10000
saved to OfflineDataset/subOptimalAgent_encConfig6_[[1, 1]].pkl
Dataset Configuration
Number of Users:        20
Window Length:          200
N_aggregation:          4
Dataflow:               thumb_bk
Random Seed:            999
Resource Bar:           5
Bandwidth:              200
Sigmoid K List:         [0.1, 0.2, 0.3, 0.4, 0.5]
Sigmoid S List:         [10.0, 10.0, 10.0, 10.0, 10.0, 10.0]
Sub Agents:             [[1, 1, 1, 1, 1]]
User Map:               [[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11], [12, 13, 14, 15], [16, 17, 18, 19]]
sub_agent_idxs: [1, 1, 1, 1, 1]
Environment Configuration
Number of Users:        4
Window Length:          200
Dataflow:               thumb_bk
N_aggregation:          4
N_r:                    5
Resource Bar:           5
Bandwidth:              60
Sigmoid K List:         [0.3]
Sigmoid S List:         [10.0]
Random Seed:            999
Environment Configuration
Number of Users:        4
Wi

Avg. packet loss rate: 0.4702335430006429
length of dataset: 10000
saved to OfflineDataset/subOptimalAgent_encConfig7_[[1, 1, 1, 1, 1]].pkl


In [ ]:
import pickle
import matplotlib.pyplot as plt

for datasetConfigIdx in [0,2]:
    datasetParams = getDatasetConfig(datasetConfigIdx)
    visualizeDatasetConfig(datasetParams)
    with open(f'OfflineDataset/subOptimalAgent_encConfig{datasetConfigIdx}_{datasetParams["sub_agents_idx"]}.pkl', 'rb') as f:
        dataset = pickle.load(f)
    print(f"Size of dataset: {len(dataset['uRecord'])}")
    print(np.mean(dataset['rewardRecord']))

Dataset Configuration
Number of Users:        8
Window Length:          200
N_aggregation:          4
Dataflow:               thumb_fr
Random Seed:            999
Resource Bar:           5
Bandwidth:              100
Sigmoid K List:         [0.3]
Sigmoid S List:         [10.0]
Sub Agents:             [[0, 0]]
User Map:               [[0, 1, 2, 3], [4, 5, 6, 7]]
Size of dataset: 10000
0.3259663531719175
Dataset Configuration
Number of Users:        8
Window Length:          200
N_aggregation:          4
Dataflow:               thumb_fr
Random Seed:            999
Resource Bar:           5
Bandwidth:              100
Sigmoid K List:         [0.1, 0.2, 0.3, 0.4, 0.5]
Sigmoid S List:         [10.0, 10.0, 10.0, 10.0, 10.0, 10.0]
Sub Agents:             [[0, 0]]
User Map:               [[0, 1, 2, 3], [4, 5, 6, 7]]
Size of dataset: 10000
0.3320853710493872
